In [4]:
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import label_binarize
from random import shuffle

In [6]:
dataset = datasets.load_iris()
iris = np.column_stack([dataset['data'], dataset['target']])
np.random.shuffle(iris)
target_multiclass = label_binarize(iris[:, -1], classes=[0, 1, 2])
print(iris)

[[ 5.5  2.5  4.   1.3  1. ]
 [ 5.4  3.9  1.3  0.4  0. ]
 [ 5.   3.5  1.6  0.6  0. ]
 [ 5.1  3.5  1.4  0.3  0. ]
 [ 4.8  3.4  1.6  0.2  0. ]
 [ 5.2  2.7  3.9  1.4  1. ]
 [ 5.6  2.9  3.6  1.3  1. ]
 [ 4.9  3.1  1.5  0.1  0. ]
 [ 6.1  2.6  5.6  1.4  2. ]
 [ 4.9  3.1  1.5  0.1  0. ]
 [ 5.   3.   1.6  0.2  0. ]
 [ 7.   3.2  4.7  1.4  1. ]
 [ 4.8  3.1  1.6  0.2  0. ]
 [ 6.8  3.2  5.9  2.3  2. ]
 [ 7.7  3.8  6.7  2.2  2. ]
 [ 4.4  3.2  1.3  0.2  0. ]
 [ 6.3  3.3  6.   2.5  2. ]
 [ 6.3  2.3  4.4  1.3  1. ]
 [ 5.8  2.8  5.1  2.4  2. ]
 [ 4.5  2.3  1.3  0.3  0. ]
 [ 5.9  3.   4.2  1.5  1. ]
 [ 5.1  3.8  1.5  0.3  0. ]
 [ 6.2  3.4  5.4  2.3  2. ]
 [ 5.2  3.5  1.5  0.2  0. ]
 [ 6.5  3.2  5.1  2.   2. ]
 [ 5.4  3.7  1.5  0.2  0. ]
 [ 5.5  4.2  1.4  0.2  0. ]
 [ 6.5  3.   5.5  1.8  2. ]
 [ 5.7  3.8  1.7  0.3  0. ]
 [ 6.5  3.   5.2  2.   2. ]
 [ 4.7  3.2  1.3  0.2  0. ]
 [ 5.   3.3  1.4  0.2  0. ]
 [ 4.9  3.   1.4  0.2  0. ]
 [ 5.8  4.   1.2  0.2  0. ]
 [ 6.9  3.2  5.7  2.3  2. ]
 [ 5.4  3.   4.5  1.

In [8]:
# Network parametrs

n_hidden_1 = 15
n_hidden_2 = 15
n_input = 4
n_classes = 3

# Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 10
display_step = 1

# tf Graph input
X = tf.placeholder('float', [None, n_input])
Y = tf.placeholder('float', [None, n_classes])

weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Create model
def perceptron(matrix):
    layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    return tf.matmul(layer_2, weights['out']) + biases['out']

# Construction
logits = perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Initializing  the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(150/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = iris[i:i+batch_size,:4], target_multiclass[i:i+batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: iris[:,:-1], Y: target_multiclass}))

Epoch: 0001 cost=111.990224202
Epoch: 0002 cost=84.211236827
Epoch: 0003 cost=73.018303426
Epoch: 0004 cost=63.062567774
Epoch: 0005 cost=53.766533915
Epoch: 0006 cost=44.354452896
Epoch: 0007 cost=34.647551600
Epoch: 0008 cost=24.629746819
Epoch: 0009 cost=14.251336797
Epoch: 0010 cost=4.285834098
Epoch: 0011 cost=2.593570264
Epoch: 0012 cost=1.000706334
Epoch: 0013 cost=0.791783217
Epoch: 0014 cost=0.737382863
Epoch: 0015 cost=0.618449079
Epoch: 0016 cost=0.572929075
Epoch: 0017 cost=0.518329751
Epoch: 0018 cost=0.455344648
Epoch: 0019 cost=0.401347922
Epoch: 0020 cost=0.347945860
Optimization Finished!
Accuracy: 0.913333
